In [ ]:
# !pip install lief

In [1]:
import pickle
# from rich.progress import track
import os
import lief
import re
import traceback
import torch
import logging

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# !unrar x /content/drive/MyDrive/dataset.zip

Streaming output truncated to the last 5000 lines.
Extracting  Dataset/Virus/Virus train/Winwebsec/048e5d06915262d056c79d9e1a07493fafcf15fc.exe      43%  OK 
Extracting  Dataset/Virus/Virus train/Winwebsec/04913aa8800b9f7311a8f68e20365ead52f6a964.exe      43%  OK 
Extracting  Dataset/Virus/Virus train/Winwebsec/0491e70e7fe39c3a653f212bd066d276d3099487.exe      43%  OK 
Extracting  Dataset/Virus/Virus train/Winwebsec/052f4f9d86a6a570b66961ece8925354c1639be5.exe      43%  OK 
Extracting  Dataset/Virus/Virus train/Winwebsec/053213fd2c45fef6dfccda41647ce26e1c71112b.exe      43%  OK 
Extracting  Dataset/Virus/Virus train/Winwebsec/053aabe348c848c4329f49b8d5f4c0f8b7307f33.exe      43%  OK 
Extracting  Dataset/Virus/Virus train/Winwebsec/054046eaf665468dad8b44230ba437847ec5963a.exe      43%  OK 
Extracting  Dataset/Virus/Virus train/Winwebsec/054b56ebf67c692011a099d1337b19e787b11fc8.exe      43%  OK 
Extracting  Dataset/V

In [6]:
malware_path = '/media/nerdcoder/New Volume/Major Project/malenv/Data/malware'
benign_path = '/media/nerdcoder/New Volume/Major Project/malenv/Data/benign'

In [7]:
os.path.exists(malware_path)

True

In [8]:
malware_pe_files = []

malware_pe_files = [os.path.join(malware_path, files) for files in os.listdir(malware_path)]
benign_pe_files = [os.path.join(benign_path, files) for files in os.listdir(benign_path)]

In [9]:
len(malware_pe_files), len(benign_pe_files)

(1247, 1851)

In [ ]:
# malware_pe_files= ['/content/drive/MyDrive/Dataset/Virus/Virus train/Locker/VirusShare_680a9dc44ef0e29e3a991151dd6c0a3b.exe']
# benign_pe_files = []

In [ ]:
def filter_imported_functions(func_string_with_library):
    """
    Filters the returned imported functions of binary to remove those with special characters (lots of noise for some reason),
    and require functions to start with a capital letter since Windows API functions seem to obey Upper Camelcase convension.

    Update: The limitation for the upper case in the preprocessing step has been removed.
    """
    func_string = func_string_with_library.split(":")[0]

    if re.match("^[a-zA-Z]*$", func_string):
        return True
    else:
        return False

In [ ]:
def process_imported_functions_output(imports):

    imports = list(filter(lambda x: filter_imported_functions(x), imports))

    return imports

In [ ]:
def extract_imports(file:str, feature_vector_mapping: dict, filtered_import_list: list,index: int = 0):

    binary = lief.parse(file)

    # imports includes the library (DLL) the function comes from
    imports = [e.name + ':' + lib.name.lower()  for lib in binary.imports for e in lib.entries]

    # preprocess imports to remove noise
    imports = process_imported_functions_output(imports)

    for lib_import in imports:

        if lib_import not in feature_vector_mapping:
            if lib_import in filtered_import_list and "hal.dll" not in lib_import:
                feature_vector_mapping[lib_import] = index
                index += 1

    return feature_vector_mapping, index

In [ ]:
def extract_sections(file, feature_vector_mapping: dict, index: int = 0):

    # debug(file)
    binary = lief.parse(file)

    sections = [section.name for section in binary.sections]

    for section in sections:
        if section not in feature_vector_mapping:
            feature_vector_mapping[section] = index
            index += 1

    return feature_vector_mapping, index

In [ ]:
feature={}
import_feature_vector = {}
section_feature_vector = {}
index = 0
import_index = 0
section_index = 0

In [ ]:
filtered_imports_file = '/content/drive/MyDrive/manipulation_content/imports_content.txt'
with open(str(filtered_imports_file), 'r') as f:
        filtered_imports = f.read()
filtered_imports_file = filtered_imports.split('\n')
# filtered_imports_file

In [ ]:
files = malware_pe_files + benign_pe_files

Import Extraction

In [ ]:
for i in track(range(len(files)), description="Extracting imports ... ", transient=True):
  file = files[i]
  try:
      feature, index = extract_imports(file,feature,filtered_imports_file, index)
      import_feature_vector, import_index = extract_imports(file,import_feature_vector,filtered_imports_file, index)
      # if not win32:
      #     os.remove(file)
      #     files.remove(file)
      # pass
  except:
      traceback.print_exc()

SECTION_INDEX = index
print(f"Import extraction completed with {SECTION_INDEX} imports... \n")

Section Extraction

In [ ]:
for i in track(range(len(files)), description="Extracting sections ...: ", transient=True):
  file = files[i]
  try:
      feature, index = extract_sections(file,feature,index)
      section_feature_vector , section_index = extract_sections(file,section_feature_vector, section_index)
      # if not win32:
      #     exception(f"\t[*] Deleting PE file : {file}")
      #     os.remove(file)
      #     files.remove(file)
      #     exception(f"\t[-] {file} has been deleted ...")
      # pass
  except:
      traceback.print_exc()
      pass
print(f"Section extraction completed with {section_index} sections ... \n")
print(f"Total size of feature vector mapping : {len(feature)} \n")

In [ ]:
# feature, index = extract_imports(file,feature,filtered_imports_file, index)
# import_feature_vector, import_index = extract_imports(file,import_feature_vector,filtered_imports_file, index)

In [ ]:
# feature, index = extract_sections(file,feature,index)
# section_feature_vector , section_index = extract_sections(file,section_feature_vector, section_index)

In [ ]:
# len(import_feature_vector) , len(section_feature_vector), len(feature)

In [ ]:
for i, import_lib in enumerate(feature):
        print(f"\t[+] feature vector value at [{i}] : {str(import_lib)}")

In [ ]:
for i, import_lib in enumerate(section_feature_vector):
  print(f"\t[+] feature vector value at [{i}] : {str(import_lib)}")

In [ ]:
for i, import_lib in enumerate(import_feature_vector):
  print(f"\t[+] feature vector value at [{i}] : {str(import_lib)}")

In [ ]:
def dumpFeatureVector(feature_vector, import_feature_vector, section_feature_vector, output_path):
    pickle.dump(feature_vector, open(os.path.join(output_path,"feature_vector_mapping.pk"), 'wb'))

    pickle.dump(import_feature_vector, open(os.path.join(output_path,"import_feature_vector_mapping.pk"), 'wb'))

    pickle.dump(section_feature_vector, open(os.path.join(output_path,"section_feature_vector_mapping.pk"), 'wb'))

In [ ]:
output_path = '/content/drive/MyDrive/Feature_Vector'
dumpFeatureVector(feature,import_feature_vector,section_feature_vector,output_path)

In [ ]:
import lief
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

def process_imported_functions_output(imports):
    imports = list(filter(lambda x: filter_imported_functions(x), imports))
    # imports = list(map(lambda x: remove_encoding_indicator(x), imports))
    return imports

def feature_generation(pe_files: list, feature_vector_mapping: dict):
    pe_files_feature_set = []

    for i in track(range(len(pe_files)), description="Generating feature vectors ... ", transient=True):
        file = pe_files[i]

        feature_vector = [0] * len(feature_vector_mapping)

        try:
            binary = lief.parse(file)
            imports = [e.name + ':' + lib.name.lower() for lib in binary.imports for e in lib.entries]
            imports = process_imported_functions_output(imports)

            sections = [section.name for section in binary.sections]

            for lib_import in imports:
                if lib_import in feature_vector_mapping:
                    index = feature_vector_mapping[lib_import]
                    feature_vector[index] = 1

            for section in sections:
                if section in feature_vector_mapping:
                    index = feature_vector_mapping[section]
                    feature_vector[index] = 1

        except Exception as e:
            logger.error(f"Error parsing {file}: {str(e)}")
            # You can choose to continue with the next file or handle the error differently

        pe_files_feature_set.append(feature_vector)

    return pe_files_feature_set


Creating feature vector with imports and sections for malware set

In [ ]:
feature_generation(malware_pe_files, feature)

In [ ]:
malware_pe_files_feature_set = torch.Tensor(feature_generation(malware_pe_files, feature))

Creating feature vector with imports and sections for Benign set

In [ ]:
benign_pe_files_feature_set = torch.Tensor(feature_generation(benign_pe_files, feature))

In [ ]:
pickle.dump(malware_pe_files_feature_set, open(os.path.join(output_path, "malware_feature_set.pk"), 'wb'))
pickle.dump(benign_pe_files_feature_set, open(os.path.join(output_path, "benign_feature_set.pk"), 'wb'))

Creating feature vector with imports for malware set

In [ ]:
malware_pe_files_import_feature_set = torch.Tensor(feature_generation(malware_pe_files, import_feature_vector))

Creating feature vector with imports for benign set

In [ ]:
benign_pe_files_import_feature_set = torch.Tensor(feature_generation(benign_pe_files, import_feature_vector))

In [ ]:
pickle.dump(malware_pe_files_import_feature_set, open(os.path.join(output_path, "malware_pe_files_import_feature_set.pk"), 'wb'))
pickle.dump(benign_pe_files_import_feature_set, open(os.path.join(output_path, "benign_pe_files_import_feature_set.pk"), 'wb'))

Creating feature vector with sections for malware set

In [ ]:
malware_pe_files_section_feature_set = torch.Tensor(feature_generation(malware_pe_files, section_feature_vector))

Creating feature vector with sections for benign set

In [ ]:
benign_pe_files_section_feature_set = torch.Tensor(feature_generation(benign_pe_files, section_feature_vector))

In [ ]:
pickle.dump(malware_pe_files_section_feature_set, open(os.path.join(output_path, "malware_pe_files_section_feature_set.pk"), 'wb'))
pickle.dump(benign_pe_files_section_feature_set, open(os.path.join(output_path, "benign_pe_files_section_feature_set.pk"), 'wb'))